In [119]:
import numpy as np
import pandas as pd
import csv
from nltk.stem import WordNetLemmatizer
import time
import re
import os

In [120]:
inds = []
for i, j in enumerate(document_ids):
    if j == 123:
        inds.append(i)

In [121]:
document_names[123]

'Expectation_propagation.txt'

In [122]:
inds

[4735, 4736, 4737, 4738]

In [148]:
sentences = []
document_ids = []

for doc_id, filename in enumerate(os.listdir('./Data/Input/collection/')):
    with open('./Data/Input/collection/' + "/" + filename, "r") as f:
        sentences_raw = '.'.join(f.read().split('\n')).split('.')

    for sentence in sentences_raw:
        sentence_nopunct = re.sub("[^A-Za-zА-Яа-я ]", '', sentence)
        sentence_nopunct = sentence_nopunct.lower().strip()
        if len(sentence_nopunct) > 1:
            sentences.append(sentence_nopunct)
            document_ids.append(doc_id)

In [149]:
document_ids[94]

0

In [150]:
sentences[94]

'where formula denote the gram matrices of formula and formula respectively formula is a transfer gram matrix defined as formula and formula'

In [157]:
syntaxnet_out = pd.read_table('./Data/Component_Results/syntaxnet/syntaxnet_out.txt', header=None,
                              dtype={0: np.int, 6: np.int},
                              quoting=csv.QUOTE_NONE, engine='c'
                             )[[0, 1, 3, 6, 7]].fillna("")

In [158]:
syntaxnet_out.rename(columns={0: "word_id", 1: "word", 3: "POS",
                              6: "parent_id", 7: "dependency"},
                     inplace=True)

In [159]:
from tqdm import tqdm

In [181]:
syntaxnet_out.loc[3350:3400]

,word_id,word,POS,parent_id,dependency,sentence_id,lemma,doc_id,doc_name
3350,6,distribution,NOUN,2,nmod,135,distribution,0,Kernel_embedding_of_distributions.txt
3351,7,under,ADP,10,case,135,under,0,Kernel_embedding_of_distributions.txt
3352,8,a,DET,10,det,135,a,0,Kernel_embedding_of_distributions.txt
3353,9,fixed,VERB,10,amod,135,fix,0,Kernel_embedding_of_distributions.txt
3354,10,value,NOUN,2,nmod,135,value,0,Kernel_embedding_of_distributions.txt
3355,11,of,ADP,12,case,135,of,0,Kernel_embedding_of_distributions.txt
3356,12,formula,ADJ,10,nmod,135,formula,0,Kernel_embedding_of_distributions.txt
3357,13,may,AUX,15,aux,135,may,0,Kernel_embedding_of_distributions.txt
3358,14,be,AUX,15,auxpass,135,be,0,Kernel_embedding_of_distributions.txt
3359,15,computed,VERB,-1,ROOT,135,compute,0,Kernel_embedding_of_distributions.txt


In [160]:
lemmatizer = WordNetLemmatizer()

syntaxnet_out["word_id"] -= 1
syntaxnet_out["parent_id"] -= 1

cur_sentence_id = -1
sentence_ids = []
lemmas = []

for row in tqdm(syntaxnet_out.itertuples(), total=len(syntaxnet_out)):
    word_id, word, pos = row.word_id, row.word, row.POS
    if word_id == 0:
        cur_sentence_id += 1

    if pos == "VERB":
        lemma = lemmatizer.lemmatize(word, pos='v')
    elif pos == "ADJ":
        lemma = lemmatizer.lemmatize(word, pos='a')
    elif pos == "ADV":
        lemma = lemmatizer.lemmatize(word, pos='r')
    else:
        lemma = lemmatizer.lemmatize(word, pos='n')

    lemmas.append(lemma)
    sentence_ids.append(cur_sentence_id)

syntaxnet_out["sentence_id"] = sentence_ids
syntaxnet_out["lemma"] = lemmas


100%|██████████| 169449/169449 [00:01<00:00, 153838.40it/s]


In [161]:
document_names = os.listdir('./Data/Input/collection/')

In [162]:
len(document_names)

198

In [163]:
sent2doc = {}
for i, doc in enumerate(document_ids):
    sent2doc[i] = doc

In [164]:
# i = 0
# doc_ids = []
# doc_names = []
# prev_sentence_id = 0
# for sentence_id in syntaxnet_out["sentence_id"]:
#     if sentence_id != prev_sentence_id:
#         i += 1
#         prev_sentence_id = sentence_id
#     doc_ids.append(document_ids[i])
#     doc_names.append(document_names[doc_ids[-1]])
# syntaxnet_out["doc_id"] = doc_ids
# syntaxnet_out["doc_name"] = doc_names

In [165]:
doc_ids = []
doc_names = []
for row in syntaxnet_out.itertuples():
    sentence_id = row.sentence_id
    doc_ids.append(sent2doc[sentence_id])
    doc_names.append(document_names[sent2doc[sentence_id]])
    
syntaxnet_out["doc_id"] = doc_ids
syntaxnet_out["doc_name"] = doc_names

In [186]:
d = dict()

prev_doc_id = 0
prev_sentence_id = 0
prev_doc_name = ""
cur_text = ""
for row in syntaxnet_out.itertuples():
    lemma = row.lemma
    sentence_id, doc_id = row.sentence_id, row.doc_id
    doc_name = row.doc_name

    if sentence_id != prev_sentence_id:
        cur_text += ' .'
        prev_sentence_id = sentence_id
    if doc_id != prev_doc_id:
#         with open(args.lemmatize + '/' + doc_name, "w") as f:
#             f.write(cur_text)
        d[prev_doc_name] = cur_text
        cur_text = ""
        prev_doc_id = doc_id
    # do not add space at start of document
    if(len(cur_text) != 0):
        cur_text += ' '
    cur_text += lemma
    prev_doc_name = doc_name
    
d[doc_name] = cur_text

In [191]:
d['Inferential_theory_of_learning.txt']

'inferential theory of learn itl be an area of machine learning which describe inferential process perform by learn agent . itl ha been develop by ryszard s . michalski in s . in itl learning process is view a a search inference through hypothesis space guide by a specific goal . result of learn need to be store in order to be use in the future'

In [190]:
syntaxnet_out.tail()

,word_id,word,POS,parent_id,dependency,sentence_id,lemma,doc_id,doc_name
169444,10,be,AUX,11,auxpass,8832,be,197,Inferential_theory_of_learning.txt
169445,11,used,VERB,6,advcl,8832,use,197,Inferential_theory_of_learning.txt
169446,12,in,ADP,14,case,8832,in,197,Inferential_theory_of_learning.txt
169447,13,the,DET,14,det,8832,the,197,Inferential_theory_of_learning.txt
169448,14,future,NOUN,11,nmod,8832,future,197,Inferential_theory_of_learning.txt


In [167]:
syntaxnet_out.head()

,word_id,word,POS,parent_id,dependency,sentence_id,lemma,doc_id,doc_name
0,0,in,ADP,1,case,0,in,0,Kernel_embedding_of_distributions.txt
1,1,machine,NOUN,2,nmod,0,machine,0,Kernel_embedding_of_distributions.txt
2,2,learning,VERB,9,csubj,0,learn,0,Kernel_embedding_of_distributions.txt
3,3,the,DET,5,det,0,the,0,Kernel_embedding_of_distributions.txt
4,4,kernel,NOUN,5,compound,0,kernel,0,Kernel_embedding_of_distributions.txt


---

In [47]:
document_names[:10]

['Kernel_embedding_of_distributions.txt',
 'AIXI.txt',
 'Explanationbased_learning.txt',
 'Algorithm_selection.txt',
 'MTheory_learning_framework.txt',
 'Overfitting.txt',
 'Cognitive_robotics.txt',
 'Movidius.txt',
 'Crossentropy_method.txt',
 'Crossvalidation_statistics.txt']

In [168]:
print(' '.join(list(syntaxnet_out[syntaxnet_out['doc_name'] == 'Kernel_embedding_of_distributions.txt']['word'])))

in machine learning the kernel embedding of distributions also called the kernel mean or mean map comprises a class of nonparametric methods in which a probability distribution is represented as an element of a reproducing kernel hilbert space rkhs a generalization of the individual datapoint feature mapping done in classical kernel methods the embedding of distributions into infinitedimensional feature spaces can preserve all of the statistical features of arbitrary distributions while allowing one to compare and manipulate distributions using hilbert space operations such as inner products distances projections linear transformations and spectral analysis this learning framework is very general and can be applied to distributions over any space formula on which a sensible kernel function measuring similarity between elements of formula may be defined for example various kernels have been proposed for learning from data which are vectors in formula discrete classescategories strings g